# 文本预处理与聚类分析

[顾思成](https://github.com/CCCCCaO) 2019.5.27

---

## 概述

爬虫是一种**数据搜集**手段，可以获取到文本，图像，音频，乃至视频等诸多数据。
而仅仅爬取到了数据是没有用的，在此基础上，我们还要对数据进行分析与解读，让数据为我所用。

文本挖掘的一般步骤：
数据搜集->文本预处理->数据挖掘与可视化->模型搭建->模型评估

在这里简单介绍一下**文本数据预处理与聚类分析**。个人拙见，还有待学习提高！

## 运行环境

+ Win10 64bit
+ Anaconda 3.7
+ Jupyter Notebook
+ Jieba
+ Scikit-learn
+ Plotly

## 1、中文分词
文本预处理的第一步，也是后续各种分析处理的基础，即分词处理。

词是人们理解语言的基础单位之一，就和我们学英语一样，你要理解某一句子，必然是建立在你理解其中大部分单词含义的基础之上的。

那么为了让计算机理解，我们也模仿人类的认识过程，也将文本切成一个个的词语，提取其中特征，来帮助后续处理。

这里采用了[Jieba中文分词组件](https://github.com/fxsjy/jieba)，另外有其他的分词组件如清华的Thulac等可选择使用。

In [ ]:
import jieba

In [ ]:
# 使用Jieba中文分词示例
seg_list = jieba.cut("我来到上海理工大学", cut_all=True)
print("【全模式】: " + "/ ".join(seg_list))         # 全模式 即完完全全地切分

seg_list = jieba.cut("我来到上海理工大学", cut_all=False)
print("【精确模式】: " + "/ ".join(seg_list))       # 精确模式 将一些应该属于一个词汇的东西合并

seg_list = jieba.cut("他来到了新媒体技术专业的学院")   # 对于未收录在语料库种的新词进行识别预测
print("【新词识别】: " + "/ ".join(seg_list))

seg_list = jieba.cut_for_search("小明本科毕业于上海理工大学，后在中科院计算所深造")  # 搜索引擎模式
print("【搜索引擎模式】: " + "/ ".join(seg_list))

我们现在假设有这么七个文本,我们试一下进行切分
1. 文本向量化是一种提取特征的操作
2. 分析文本应该关注文本的特征
3. 床前明月光
4. 苹果是水果中的一种,深受人们喜爱
5. 今年下半年中美合拍的西游记
6. 我最爱吃苹果
7. 苹果是我的最爱

In [ ]:
corpus=["文本向量化是一种提取特征的操作",
       "分析文本应该关注文本的特征",
       "床前明月光",
        "苹果是水果中的一种深受人们的喜爱",
       "今年下半年中美合拍的西游记",
       "我最爱吃苹果",
       "苹果是我的最爱"]

for i in range(len(corpus)):
    seg_list = jieba.cut(corpus[i], cut_all=False)
    print(i+1, ": ", "/ ".join(seg_list))

---

## 2、文本特征处理
现在我们已将文本切分成一堆的词组成的东西，但是对于计算机而言还是无法理解。我们要对这些词特征，再进行特征提取，将词汇以及其中的联系关系转换成数值，这样计算机才能够认识它。特征提取有向量化与Hash Trick等，而向量化又是最为常用的。

文本向量化即是将文本变成一个向量，好比用某个坐标点来标记某个文本。文本向量化的方法有很多，例如最简单的词频统计，按照某个词在这篇文章中出现的频率作为这篇文章的特征向量。而这里我们用TF-IDF值。TF-IDF值是对词频的一种修正改善，减小了某些出现次数很多但又没有意义的词如“的”，“了”等词的影响。
推荐阅读[文本挖掘之TF-IDF值](https://www.cnblogs.com/pinard/p/6693230.html)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 示例文本
corpus=["文本 向 量化 是 一种 提取 特征 的 操作",
       "分析 文本 应该 关注 文本 的 特征",
       "床 前 明月光",
        "苹果 是 水果 中 的 一种 深受 人们 的 喜爱",
       "今年 下半年 中 美 合拍 的 西游记",
       "我 最 爱 吃 苹果",
       "苹果 是 我 的 最爱",]
tfidf = TfidfVectorizer()                                      # 生成TFIDF对象
re = tfidf.fit_transform(corpus)                               # 用TFIDF拟合并标准化

# 结果输出
wordlist = tfidf.get_feature_names()                           #获取词袋模型中的所有词  
weightlist = re.toarray()                                      # TF-IDF的矩阵 元素re[i][j]表示j词在i类文本中的TF-IDF权重
for i in range(len(weightlist)):  
    print ("-------第",i+1,"段文本的词语tf-idf权重------")  
    for j in range(len(wordlist)):  
        print (wordlist[j],weightlist[i][j])  

In [ ]:
print(weightlist)

---

## 3、聚类分析

对于我们而言，是一句或一段话，而现在，经过上面的向量化，对于计算机而言，其实就是一串数字组成的数据，即一个矩阵。那么这些数据之间有什么联系？如何去分析呢？那么就需要对文本建立数学模型，进行解读与分析。方法有很多种：文本摘要，关键词抽取，文本聚类，文本分类，文档主题模型，观点抽取，情感分析等等。

人类在研究事物的过程中，常将东西分门别类来观察研究。而将物理或抽象对象的集合分成由类似的对象组成的多个类的过程被称为聚类，最直观理解即是：“物以类聚，人以群分”。通过聚类分析我们可以使得数据变量中的一些关联性的特征呈现出来，找到数据之间的相似性，相似的放在一类里面。

需要注意的是分类和聚类是不一样的，
>最直白的理解，聚类就是给你一堆东西，然后分成几类。分类就是给你几类东西，再给你一个。然后这一个属于哪一类。 --知乎的Tavion Fu

聚类的方法有很多种，DBSCAN,DPEAK等等，这里介绍一下K-means聚类。

K-means聚类算法的过程：

1. 在所有数据中随机选取K个中心点
2. 分别计算每个点到每个中心点的距离，对于每个点来说选取距离其最近的那个中心点，归为一类。
3. 重新计算K个中心点，即对该类所有点的坐标求均值means，得到新的中心点
4. 重复第二步，直至中心点变化极小为止

推荐阅读[K-means聚类算法 JerryLead博客](https://www.cnblogs.com/jerrylead/archive/2011/04/06/2006910.html)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# 使用Kmeans聚类 假设有4类
n_clusters = 4
kmeans = KMeans(n_clusters)
kmeans.fit(weightlist) 


对于下面这7个文本而言
1. 文本向量化是一种提取特征的操作
2. 分析文本应该关注文本的特征
3. 床前明月光
4. 苹果是水果中的一种,深受人们喜爱
5. 今年下半年中美合拍的西游记
6. 我最爱吃苹果
7. 苹果是我的最爱

In [ ]:
# 结果输出
label_all = []
for index, label in enumerate(kmeans.labels_, 1):
    label_all.append(label)
    print("文本序号 {} , 是第 {} 类".format(index, label))

---

## 4、可视化

其实我们已经得到分类的结果了，如上面输出的那样。可是这不直观，还需要你去上下结合着看，当文本数量激增的时候，非常头疼，这时候我们需要进行可视化的操作，使得结果更浅显易懂，直观生动。

In [ ]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=False)
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

In [ ]:
# 使用T-SNE进行降维 将高维空间的TF-IDF矩阵降低到2维平面 并标准化
tsne = TSNE(n_components=2, init='random')                   
decomposition_data = tsne.fit_transform(weightlist)     
X = StandardScaler().fit_transform(decomposition_data)    

In [ ]:
# 使用Plotly进行可视化操作
text = [index for index, label in enumerate(kmeans.labels_, 1)]

trace = [go.Scatter(
        x=X[:, 0], 
        y=X[:, 1],
        text=text,
        textposition='bottom center', 
        marker=dict(
            symbol="circle",
            size = 10,
            color=kmeans.labels_,  # 每个点的颜色是它聚类后的类别
            opacity=0.8,
            colorscale='Jet',
            showscale=True
        ),
        mode='markers'
    )]
title = '7篇文本的K-Means聚类 K=' + str(n_clusters)
layout = go.Layout(title=title) 
fig = go.Figure(data=trace, layout=layout)

plotly.offline.iplot(fig)

In [ ]:
from sklearn.datasets import make_blobs
# 生成一组数据
n_samples = 1500
random_state = 170
X_varied, y_varied = make_blobs(n_samples=n_samples,
                                cluster_std=[1.0, 2.5, 0.5],
                                random_state=random_state)
# Kmeans聚类
n_clusters = 5
y_pred = KMeans(n_clusters, random_state=random_state)
y_pred.fit_predict(X_varied)

# 绘图
trace = [go.Scatter(
        x=X_varied[:, 0], 
        y=X_varied[:, 1],
        textposition='bottom center', 
        marker=dict(
            symbol="circle",
            size = 10,
            color=y_pred.labels_,
            opacity=0.8,
            colorscale='Jet',
            showscale=True
        ),
        mode='markers'
    )]
title = 'K-Means聚类示例 K=' + str(n_clusters)
layout = go.Layout(title=title) 
fig = go.Figure(data=trace, layout=layout)

plotly.offline.iplot(fig)